In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import numpy as np
from multiprocessing import Pool, cpu_count

In [3]:

def lorenz(x, y, z, sigma=10.0, rho=28.0, beta=8.0/3.0):
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return dx, dy, dz

def euler(step, x, y, z):
    dx, dy, dz = lorenz(x, y, z)
    x = x + step*dx
    y = y + step*dy
    z = z + step*dz
    return x, y, z

def runge_kutta_2rd_order(step, x, y, z):
    # k1
    dx1, dy1, dz1 = lorenz(x, y, z)
    
    # k2
    dx2, dy2, dz2 = lorenz(x + 0.5*step*dx1, y + 0.5*step*dy1, z + 0.5*step*dz1)

    # Compute new values
    x = x + step*dx2
    y = y + step*dy2
    z = z + step*dz2

    return x, y, z

def runge_kutta_3rd_order(step, x, y, z):
    # k1
    dx1, dy1, dz1 = lorenz(x, y, z)
    
    # k2
    dx2, dy2, dz2 = lorenz(x + 0.5*step*dx1, y + 0.5*step*dy1, z + 0.5*step*dz1)
    
    # k3
    dx3, dy3, dz3 = lorenz(x + 0.75*step*dx2, y + 0.75*step*dy2, z + 0.75*step*dz2)
    
    # Compute new values
    x = x + step * (2*dx1 + 3*dx2 + 4*dx3) / 9.0
    y = y + step * (2*dy1 + 3*dy2 + 4*dy3) / 9.0
    z = z + step * (2*dz1 + 3*dz2 + 4*dz3) / 9.0
    
    return x, y, z

def runge_kutta_4th_order(step, x, y, z):
    dx1, dy1, dz1 = lorenz(x, y, z)
    
    dx2, dy2, dz2 = lorenz(x + 0.5*step*dx1, y + 0.5*step*dy1, z + 0.5*step*dz1)
    
    dx3, dy3, dz3 = lorenz(x + 0.5*step*dx2, y + 0.5*step*dy2, z + 0.5*step*dz2)
    
    dx4, dy4, dz4 = lorenz(x + step*dx3, y + step*dy3, z + step*dz3)
    
    x = x + step * (dx1 + 2.0*dx2 + 2.0*dx3 + dx4) / 6.0
    y = y + step * (dy1 + 2.0*dy2 + 2.0*dy3 + dy4) / 6.0
    z = z + step * (dz1 + 2.0*dz2 + 2.0*dz3 + dz4) / 6.0
    
    return x, y, z


In [11]:
def simulate_lorenz(steps=1000, step_size=0.02, x0=2.0, y0=1.0, z0=1.0):
    xs, ys, zs = [], [], []
    x, y, z = x0, y0, z0

    for _ in range(steps):
        x, y, z = runge_kutta_4th_order(step_size, x, y, z)
        xs.append(x)
        ys.append(y)
        zs.append(z)
        
    return xs, ys, zs


xs, ys, zs = simulate_lorenz(x0=2, y0=1, z0=1)
# print the 10 first values
print(f'x: {xs[:10]}')
print(f'y: {ys[:10]}')
print(f'z: {zs[:10]}')

x: [1.9150883377414816, 2.027129176297306, 2.3074577756956183, 2.748348661130485, 3.3575424262247524, 4.154644709755155, 5.168082054405992, 6.431087483759159, 7.974567872895745, 9.813766245208152]
y: [2.016698257594035, 3.0216722677319208, 4.1095468416151855, 5.361521869006261, 6.852667668371596, 8.6544149139586, 10.830820640595206, 13.425025341253347, 16.42965665621837, 19.73330710237254]
z: [1.0052436712807462, 1.0492631216002608, 1.1446860607260092, 1.3176644505240072, 1.61144641073042, 2.0938042734378928, 2.868178034723468, 4.087843970550031, 5.969526855609691, 8.79559165978617]


In [5]:
def render_frame(args):
    num, xs, ys, zs = args
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')  # Remove the box frame
    
    # Set the background color to black
    ax.set_facecolor('black')
    fig.set_facecolor('black')
    
    # Remove the grid
    ax.grid(False)
    
    # Remove the axis ticks and labels
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    ax.xaxis.line.set_color((1.0, 1.0, 1.0, 0.0))  # Make x axis invisible
    ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))  # Make y axis invisible
    ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))  # Make z axis invisible
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))  # Make x axis transparent
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))  # Make y axis transparent
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))  # Make z axis transparent
    
    # Create a gradient based on the length of xs up to num
    colors = plt.cm.viridis(np.linspace(0, 1, num))
    
    for i in range(1, num):
        ax.plot(xs[i-1:i+1], ys[i-1:i+1], zs[i-1:i+1], color=colors[i], lw=2)
    
    ax.set_xlim(min(xs), max(xs))
    ax.set_ylim(min(ys), max(ys))
    ax.set_zlim(min(zs), max(zs))
    
    filename = f"frames/frame_{num:04d}.png"
    plt.savefig(filename, dpi=140, edgecolor='none', pad_inches=0, facecolor=fig.get_facecolor())
    plt.close(fig)
    return filename

def animate_lorenz(xs, ys, zs, frames_step=24):
    frames = list(range(0, len(xs), frames_step))
    with Pool(cpu_count()) as pool:
        # Use starmap or map depending on your inputs
        filenames = pool.map(render_frame, [(num, xs, ys, zs) for num in frames])
    return filenames

# # Simulate
# xs, ys, zs = simulate_lorenz()

# Animate and get filenames of generated frames
import os
os.makedirs("frames", exist_ok=True)
print(f"xs.shape={len(xs)}, ys.shape={len(ys)}, zs.shape={len(zs)}")
# filenames = animate_lorenz(xs, ys, zs)
# print(f"Generated {len(filenames)} frames: ", filenames)


# os.system("ffmpeg -framerate 24 -pattern_type glob -i 'frames/*.png' \
        # -c:a copy -shortest -c:v libx264 -pix_fmt yuv420p out10fps.mp4")


xs.shape=1000, ys.shape=1000, zs.shape=1000
